<h1><center>Labor Economics</center></h1>

### Exam
- project work:
    - by yourself or in a group
    - presentation at the end of the term
    - brief written summary of your work, 3-5 pages

### Exercise class
- biweekly
- working on data
- small exercises
- working on your own project

### Tools
- Stata
<br>Feel free to use any code you like for your own analysis (R, SAS, Stata, Python, ...). However, since we had to decide
on a tool to be used in this class, I am going to work with Stata.

### Data
- Current Population Survey (U.S.)
- CPS-Supplements

## Stata

<b>getting started</b>
-   https://ell.stackexchange.com/questions/91623/difference-between-by-and-via <br>List of helpful links for beggining to work with stata <br>

- https://www.stata.com/features/ <br>List of features in Stata <br>

- vast amount of literature and tutorials available online <br>




## Current Population Survey

- <a href="https://www.census.gov/programs-surveys/cps.html">cps homepage</a>
- started in 1940, open data, free access
- monthly survey of ~ 60.000 U.S. households
- participants: over 15 years old, not part of the armed forces, not working in institutions such as prisons, long-term care hospitals and nursing homes
- conducted by United States Census Bureau, commissioned by Bureau of Labor Statistics

- questionnaire covers demographic and economic (especially labor force) topics
- expanded by regular (at least yearly) supplements, such as
    - Social and Economic Supplement
    - Computer and Internet Use
    - Tobacco Use
    - Civic Engagement
    - ...
    
    (complete list of supplements: <a href = https://www.census.gov/programs-surveys/cps/about/supplemental-surveys.html>LINK</a>)

- process of data collection (monthly data):
    - households are interviewed in the week of the 19th of a month
    - revisited for three following months
    - revisited again in the next year and the very same four months
    
<center><b>4-8-4 pattern</b></center>

## IPUMS

- problem: CPS solely assigns unique identifiers to households, not to individuals
    - i.e. the same household is revisited seven times - regardless of which family or individual answers the door
- longitudinal research on an individual level complicated, but not impossible

- linking items via matching making use of variables such as household identifier, line number within household and demographic characteristics (e.g. sex, age, race) <br>
    - <i>Drew, J.A., Flood, S. and Warren, J.R. (2014) "Making full use of the longitudinal design of the Current Population Survey:
    Methods for linking records across 16 months", Journal of Economic and Social Measurement, 39, pp. 121-144. <br> DOI 10.3233/JEM-140388</i>
- accordingly prepared microdata available in the <b>Integrated Public Use Microdata Series (IPUMS-CPS)</b> <a href="https://cps.ipums.org/cps/">LINK</a>

## <span style="color:orange">Homework</span>

Visit https://cps.ipums.org/cps/ and create an account. Register with any valid e-mail adress, preferably your uni-mail adress. Select all the variables and samples potentially relevant to your project.
Use the desciprtion of the variables to decide which to include in your extract. Try to minimise the number of variables in your extract.
Submit your request. It may take some time for your extract to be prepared and ready for download. The download link will be provided to you via e-mail once the data is processed.


Hint #1: If you already stumbled upon research papers dealing with questions similar to your project and working with CPS-data, you may include the author's choice of variables in your set as well.

Hint #2: take your time. This task is not trivial. Your work depends on the data at hand. Carefully chosen variables suited for answering a particular question facilitate your research.

- We confine our analysis to the years from 2000 onwards
- Choose your variables wisely: The number of variables inflates the size of the data set. E.g. 80 variables across monthly data between 2000 and 2019 constitute more than 7 gb of data
- adding supplements you receive data sets as large as 50 gb and more

- the bigger your extract,
    - the more time it takes the IPUMS server to process your request (from several hours to possibly several days)
    - the more time it takes you to download it
    - the more time and processing power is required to modify and analyse the data
- Good practice: Select cases to reduce the size, drop redundant variables (e.g. recodes), do not include every supplement (since for most individuals in the monthly survey the values are blank anyway)


## <span style="color:orange">Homework</span>

Validate the CPSID linkages following the hints in the description of the variable. (<a href="https://cps.ipums.org/cps-action/variables/cpsidp#description_section">LINK</a> to description)

It is recommended to verify the linkages by checking the consistency of the variables AGE, RACE and SEX.
For a linkage to be valid, a CPSID should have a unique gender and race. Though age is not a static variable, it should be in a certain range for the survey lifetime of an individual, i.e. between $AGE_{i, t=1} = AGE_{i, t=0} + d$ with $d \in [0, 2]$.

Hint: This is not an easy task to do. You might come up with an own approach outside of Stata, you might be able to write proper code in Stata.
Furthermore, you have to consider whether you want to check the linkages for two consecutive months only or if you want to check the whole survey lifetime of a participant.

Here is my approach for validating the linkages between two consecutive months:

In [ ]:
* loading data
cd "C:\Users\Hannes\Documents"
use "session01.dta", clear

* Creating lead variables
bysort cpsidp (year month mish): gen lead_mish = mish[_n+1]
bysort cpsidp (year month mish): gen lead_age = age[_n+1] if lead_mish - mish == 1
bysort cpsidp (year month mish): gen lead_race = race[_n+1] if lead_mish - mish == 1
bysort cpsidp (year month mish): gen lead_sex = sex[_n+1] if lead_mish - mish == 1

In [ ]:
* Creating lead variables
bysort cpsidp (year month mish): gen lag_mish = mish[_n-1]
bysort cpsidp (year month mish): gen lag_age = age[_n-1] if mish - lag_mish == 1
bysort cpsidp (year month mish): gen lag_race = race[_n-1] if mish - lag_mish == 1
bysort cpsidp (year month mish): gen lag_sex = sex[_n-1] if mish - lag_mish == 1

In [ ]:
* validation
gen linked_f = .m
replace linked = 1 if lead_mish - mish == 1 & lead_race == race & lead_sex == sex & ((mish == 4 & lead_age - age >= 0 & lead_age - age <= 2) | (mish ~= 4 & lead_age - age >= 0 & lead_age - age <= 1))

gen linked_b = .m
replace linked_b = 1 if mish - lag_mish == 1 & race == lag_race & sex == lag_sex & ((mish == 5 & age - lag_age >= 0 & age - lag_age <= 2) | (mish ~= 5 & age - lag_age >= 0 & age - lag_age <= 1))

In [ ]:
* flag for linked backwards and/or forwards
gen linked = linked_f ==1 | linked_b == 1

In [ ]:
tab linked

What makes the linking invalid for those 197,5 k observations?

In [ ]:
gen age_diff = lead_age - age
tab age_diff if linked == 0 & !inrange(age_diff, 0, 2)

In [ ]:
numlabel, add
tab sex lead_sex if linked == 0 & sex ~= lead_sex

In [ ]:
tab race lead_race if linked == 0 & race ~= lead_race

In [ ]:
by cpsidp: egen maxmish = max(mish)
tab maxmish

In [ ]:
egen tag_help = tag(cpsidp month year)
egen obs = total(tag_help), by(cpsidp)
drop tag_help

tab obs if obs == 1 & obs ~= maxmish


In [ ]:
*gen mish_delta = lead_mish - mish
tab mish_delta if mish_delta ~= 1 & linked == 0

So why do we make such an effort looking at invalid linkages?

Because you ought to know the data you are about to dismiss as well as the data you would like to workt with.
Checking you data for sanity might reduce the risk of measurement errors (wrong observations).
Apparently the concept of CPSIDP is not as trustworthy as one would have thought beforehand.

<center>Now that we have our data set, we can start working with it.</center>

# Insight & descriptive statistics (Task 1)

### How many unique respondents are introduced into the panel each year? 

In [ ]:
count unique new individuals by year
sort year month
egen tag_help = tag(cpsidp)
egen num_entries = total(tag_help), by(year)
tabdisp year, c(num_entries)

### What is the average yearly share of female respondents overall? How does the average annual share evolve from 2000-2020?

In [ ]:
* share of female responses

summarize sex
replace sex = sex - 1
label define sex 0 "male" 1 "female"
label value sex sex

* share overall
mean sex

* share per year
bysort year: egen fem = mean(sex)
table year, content(max fem)
* or 
tab year sex, row

If you trying to calculate the share of a variable, yet it is not binary coded, you can try different approaches.

tab gives you pretty much everything you want:

>tab year sex, row

If you want to store the share as a ne variable, you have to come up with something different.

Let us generate a variable for the share of unique female respondents:

In [ ]:
egen fem_counter = tag(year cpsidp) if sex == 1
egen fem_total = total(fem_counter), by(year)
egen both_counter = tag(year cpsidp)
egen respondents_total = total(both_counter), by(year)
gen fem_share = fem_total / respondents_total

In [ ]:
save "workfile01.dta"

In [ ]:
collapse (max) fem_share, by(year)
twoway (line fem_share year, sort), ytitle(share) xtitle(year) title(Share of female respondents) yscale(range(0.00 1.00)) ylabel(0(0.1)1)

In [ ]:
use "workfile01.dta", clear

I strongly recommend to drop the auxiliary after that, so that the table stays clear and tidy.

In [ ]:
drop fem_* both_* respondents_*

### What is the average yearly share of respondents holding at least a college degree overall? How does the average annual share evolve from 2001-2020?

For the average share of respondents holding a college degree you have to calculated the share of respondents instead the share of responses. You also have to keep in mind that individuals are paticipating up to four times a year in the survey.
The following code considers this as well as the fact that a person can obtain college degree between the waves of interviews. The highes degree obtained within a year will be counted.

In [ ]:
tab educ
gen college = .
replace college = 1 if inlist(educ, 125, 124, 092, 123, 111)

egen tag_college = tag(year cpsidp college)
egen tag_respondents = tag(year cpsidp)

egen college_total = total(tag_college), by(year)
egen respondents_total = total(tag_respondents), by(year)

gen college_share = college_total / respondents_total

In [ ]:
save "workfile01.dta", replace

In [ ]:
collapse (max) college_share, by(year)
twoway (line college_share year, sort), ytitle(share) xtitle(year) title(Share of respondents holding at least a college degree) yscale(range(0.00 1.00)) ylabel(0(0.1)1)

In [ ]:
use "workfile01.dta", clear

In [ ]:
drop college* respondents*

### Plot the distribution of hourly wages for the years 2001, 2009 and 2018.

In [ ]:
mvdecode(hourwage), mv(999.99)

qui histogram hourwage if year==2001, name(h2001) yscale(range(0 1)) ylabel(0(0.02)0.1)
qui histogram hourwage if year==2009, name(h2009) yscale(range(0 1)) ylabel(0(0.02)0.1)
qui histogram hourwage if year==2018, name(h2018) yscale(range(0 1)) ylabel(0(0.02)0.1)
graph combine h2001 h2009 h2018, col(3)

### Is the data representative of the distribution of the population across all 50 U.S. states in 2017?

In [ ]:
tab statecensus

In [ ]:
egen distinct2017 = tag(cpsidp year) if year == 2017
tab statecensus if distinct2017 == 1

Compare the frequencies with official figures, which you can finde here:
<a href=https://www.census.gov/data/tables/time-series/demo/popest/2010s-state-total.html>census.gov</a>

### You have the task to create statistics of employment status, occupation, wage and education. <br>You are also asked to do this analysis for the subpopulations defined by age, race, sex and citizenship.<br> How would you proceed?

The idea of this task is to rise awareness of the different scale of measurements present in this data set.
E.g., there is no point in calculating 5 point statistics for a nominal variable. Frequency tables are a helpful start for a data analysis.

There are different ways in stata to create meaningful statistics. When doing so for various subpopulations, you should avoid to render the statistics to granular, i.e. analsing too many groups. The more result tables you have, the harder it gets to compare them.


In [ ]:
/* There exist a variety of commands that yield a quick overview of the data as well as some related basic statistics
and further information.*/
describe

In [ ]:
codebook

In [ ]:
tab year

In [ ]:
summarize age, detail

In [ ]:
// Create frequency tables using the tabulate command
tabulate educ sex, row

In [ ]:
numlabel, add
tab educ

Value 92 represents associate's degrees. Those degrees academic degrees in the US and in other countries who introduced it. However, in most countries of the EU those degrees are not considered academic.
Now we have to decide whether to label those degrees as academic college degrees or not.
I choose not to do so, since the results of my analysis would be hard to compare with the situation in other countries where such a degree is unknown.

If you happen to work with this variable in your project, you may decide to label associate's degrees differently. However you proceed, please elaborate on your decision.

In [ ]:
/* create new variable college that is 0 for every degree below college, 1 for every degree equivalent
or above college and .m for missing values */

gen college = 0
replace college = 1 if educ > 92
replace college = .m if educ == 1

In [ ]:
tab sex college, row

In [ ]:
bysort sex college: tab occ empstat, row

In [ ]:
numlabel, add
tabulate faminc

In [ ]:
mvdecode(faminc), mv(996 997 999)

In [ ]:
tabstat faminc, by(year) statistics(p25, median, p75)

In [ ]:
graph bar (count), over(faminc) over(year, label(angle(v))) asyvars stack percent

# Effects of wage on hours worked (intensive margin) [Task 2]

In [ ]:
*load data
cd "C:\Users\Hannes\Documents"
use "session01.dta", clear

In [ ]:
gen hwage_r = round(hourwage, 1)

In [ ]:
tab hwage_r

<a href="https://cps.ipums.org/cps-action/variables/HOURWAGE#description_section"> IPUM's description of hourwage:</a>
>HOURWAGE reports how much the respondent earned per hour in the current job, for those workers paid an hourly wage (and coded as "2" in PAIDHOUR). Amounts are expressed as they were reported to the interviewer; users must adjust for inflation using Consumer Price Index adjustment factors. Researchers should use the EARNWT weight with this variable. <br> <br> Users should note that HOURWAGE originally had two implied decimal places, but was revised so that the command files provided by IPUMS divide HOURWAGE by 100. <br> <br> HOURWAGE is one of the <a href = "https://cps.ipums.org/cps/outgoing_rotation_notes.shtml">Outgoing Rotation/Earner Study</a> questions.

In [ ]:
summarize hourwage, detail


The 10th percentile is 999.99, which means "not in universe". Thus we are lacking information on hourly wage for at least 90% of the respondents.
The variable for wages in topcoded for the sake of discretion: Every wage that would result in a yearly income of more than \$ 100,000 (1985-2002) and \\$ 150,000 (since 2003) is coded as 99.99 in hourwage.

In [ ]:
mvdecode(hourwage), mv(999.99)

In [ ]:
summarize hourwage, detail

So how many topcoded values do we have in hourwage?

In [ ]:
tab hourwage if hourwage == 99.99

In [ ]:
sum hourwage if hourwage < 99.99

In [ ]:
mvdecode(hourwage), mv(99.99)

In [ ]:
tab month if !missing(hourwage)

### Examine the effect of wage on hours worked (intensive margin of labour supply) using an OLS.

In [ ]:
numlabel , add
tabulate uhrswork1

uhrswork1 = 999 denotes missing values. But how do we handle varying hours (i.e. uhrswork1=997)? In a way we are lacking information here as well.

In [ ]:
tab empstat

From our understanding individuals with $empstat \in [1, 12]$ should work at least some hours a week.

In [ ]:
tab uhrswork1 if inrange(empstat, 1, 12)

uhrswork1 = 997 affects approximately 7.6 % of all respondents having a job. We are going to exclude 997 an 999 from the analysis, since we are interested in effects on the intensive margin.
Keep in mind that the missing of data (or in our case a varying in hours) could yield some important implications as well.

To elaborate on that, let us have a brief look at missing values.

#### Missing Completely At Random (MCAR)
The occurence of a missing value in Y does not depend on realisations of any other variable $X_i$ nor does it depend on other realisations of Y itself.<br>
e.g.
- transmission errors

#### Missing At Random (MAR)
The occurence of a missing value can be fully explained by the other variables $X_i$.<br>
e.g.
- males are less likely to state existing mental problems - regardless of whether they have mentral problems or not

#### Missing Not At Random (MNAR)
The occurence of a missing value in Y does depend on realisations of Y itself and can also depend on realisations of other variables $X_i$.<br>
e.g.
- People with high income are less likely to state their income.



Keep in mind that the missing of data might hold some information as well.

In [ ]:
mvdecode(uhrswork1), mv(999 997)

In [ ]:
reg uhrswork1 hourwage [pweight=earnwt]

OLS minimizes
$$\sum_{i=1}^n (y_i - (\alpha1 + \beta x_i))^2$$
Introducing a weight $w_i$ (probability weight) for every observation:
$$\sum_{i=1}^n w_i^2 (y_i - (\alpha1 + \beta x_i))^2$$
$$\sum_{i=1}^n (w_iy_i - (\alpha w_i + \beta w_i x_i))^2$$

### Determine the correlation between wages and hours worked. What is the difference between OLS and correlation?

In [ ]:
corr uhrswork1 hourwage

The default correlation command does not feature pweights. However, for point estimates of the correlation it is recommended to use aweight instead - though it delivers bogus p-values.
https://www.stata.com/support

In [ ]:
corr uhrswork1 hourwage [aweight=earnwt]
display round(r(rho)^2, .0001)

### Add age, sex and citizenship to your model. Include the log of wage and hours. Interpret the results of a multivariate regression.